In [7]:
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torch.autograd import Variable
import pathlib
from torchvision.transforms import transforms
from PIL import Image
from io import open
import cv2
import torch.functional as F
from torchvision.models import squeezenet1_1
import torchvision

In [18]:
train_path = '/home/fasal/projects/pytorch/FirstProject/Dataset/seg_train/seg_train'
pred_path = '/home/fasal/projects/pytorch/FirstProject/Dataset/seg_pred/seg_pred'

In [19]:
root = pathlib.Path(train_path)
classes = sorted([j.name.split('/')[-1] for j in root.iterdir()])
print(classes)

['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


In [20]:
#Cnn network
class ConvNet(nn.Module):
    def __init__(self,num_classes = 6,**kwargs):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s)+1
        
        #Input shape(256,3,150,150)
        
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape(256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape(256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape(256,12,150,150), TO BRING THE NON-LINIEARITY
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Shape(256,12,75,75), TO REDUCE THE HIGHT AND WIDTH
        
        self.conv2 = nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape(256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape(256,20,75,75)
        
        self.conv3 = nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape(256,32,75,75)
        self.bn2=nn.BatchNorm2d(num_features=32)
        #Shape(256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape(256,32,75,75)
        
        self.fc=nn.Linear(in_features = 32*75*75,out_features = num_classes)
        
        
        #Feed forward functions
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn2(output)
        output=self.relu3(output)
            
        #Above output will be in matrix form, with shape(256,32,75,75)
            
        output=output.view(-1,32*75*75) # IN ORDER TO FEED IT INTO A FULLY CONNECTED LAYER, RESHAPE IT
            
        output=self.fc(output)
            
        return output

In [21]:
checkpoint = torch.load('best_checkpoint.model')
model = ConvNet(num_classes=6)
model.load_state_dict(checkpoint)
model.eval()

ConvNet(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (conv3): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (fc): Linear(in_features=180000, out_features=6, bias=True)
)

In [22]:
transformer = transforms.Compose([
    transforms.Resize((150,150)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],
                         [0.5,0.5,0.5])
]) 

In [23]:
#prediction function

def prediction(img_path,transformer):
    
    image=Image.open(img_path)
    image_tensor=transformer(image).float()
    image_tensor=image_tensor.unsqueeze_(0)
    
    if torch.cuda.is_available():
        image_tensor.cuda()
    input=Variable(image_tensor)
    
    output=model(input)
    index=output.data.numpy().argmax()
    pred=classes[index]
    return pred

In [27]:
images_path=glob.glob(pred_path+'/*.jpg')

In [30]:
pred_dict={}

for i in images_path:
    pred_dict[i[i.rfind('/')+1:]]=prediction(i,transformer)

In [31]:
pred_dict

{'20855.jpg': 'forest',
 '18071.jpg': 'street',
 '23953.jpg': 'street',
 '17899.jpg': 'mountain',
 '18869.jpg': 'street',
 '23946.jpg': 'street',
 '21171.jpg': 'glacier',
 '17186.jpg': 'sea',
 '5128.jpg': 'mountain',
 '22507.jpg': 'street',
 '23788.jpg': 'buildings',
 '1418.jpg': 'buildings',
 '13712.jpg': 'street',
 '6376.jpg': 'street',
 '10748.jpg': 'street',
 '15105.jpg': 'street',
 '20669.jpg': 'sea',
 '21628.jpg': 'glacier',
 '5016.jpg': 'glacier',
 '23732.jpg': 'glacier',
 '24067.jpg': 'street',
 '5901.jpg': 'forest',
 '4422.jpg': 'street',
 '858.jpg': 'mountain',
 '13977.jpg': 'mountain',
 '24253.jpg': 'buildings',
 '15336.jpg': 'street',
 '11549.jpg': 'glacier',
 '15075.jpg': 'street',
 '13065.jpg': 'glacier',
 '22181.jpg': 'forest',
 '10083.jpg': 'glacier',
 '19188.jpg': 'forest',
 '9131.jpg': 'buildings',
 '21899.jpg': 'street',
 '826.jpg': 'buildings',
 '16886.jpg': 'buildings',
 '8726.jpg': 'street',
 '22190.jpg': 'forest',
 '12895.jpg': 'forest',
 '14511.jpg': 'buildings'